In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [37]:
device = torch.device('cpu')

class NN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(NN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.nn_layer = nn.Linear(16*7*7, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.nn_layer(x)
        return x

In [38]:
learning_rate = 0.01
batch_size = 100
epochs = 10

model = NN(in_channels=1, num_classes=10).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [39]:
train_dataset = datasets.MNIST(root='Data/MNIST-Dataset', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='Data/MNIST-Dataset', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [40]:
model.train()

for epoch in range(epochs):
    num_samples = 0
    num_correct = 0
    for batch_idx, (data, targets) in enumerate(train_loader):

        data = data.to(device=device)
        targets = targets.to(device=device)
        
        predicted_output = model(data)
        
        _, prediction = predicted_output.max(1)
        num_samples += predicted_output.size(0)
        num_correct += (prediction==targets).sum()
        
        loss = loss_function(predicted_output, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if ((batch_idx+1) % 200 == 0):
            print(f'Batch {batch_idx+1}: Accuracy: {num_correct / num_samples * 100.00}')
    print(f'Epoch {epoch+1} completed with Accuracy {num_correct / num_samples * 100.00}')
    print()
print()
print("Training Completed!")
   

Batch 200: Accuracy: 91.73999786376953
Batch 400: Accuracy: 94.19999694824219
Batch 600: Accuracy: 95.34500122070312
Epoch 1 completed with Accuracy 95.34500122070312

Batch 200: Accuracy: 97.875
Batch 400: Accuracy: 98.02999877929688
Batch 600: Accuracy: 98.04499816894531
Epoch 2 completed with Accuracy 98.04499816894531

Batch 200: Accuracy: 98.47000122070312
Batch 400: Accuracy: 98.40249633789062
Batch 600: Accuracy: 98.37667083740234
Epoch 3 completed with Accuracy 98.37667083740234

Batch 200: Accuracy: 98.7750015258789
Batch 400: Accuracy: 98.6875
Batch 600: Accuracy: 98.68333435058594
Epoch 4 completed with Accuracy 98.68333435058594

Batch 200: Accuracy: 98.69999694824219
Batch 400: Accuracy: 98.70999908447266
Batch 600: Accuracy: 98.67666625976562
Epoch 5 completed with Accuracy 98.67666625976562

Batch 200: Accuracy: 99.17500305175781
Batch 400: Accuracy: 98.9124984741211
Batch 600: Accuracy: 98.81832885742188
Epoch 6 completed with Accuracy 98.81832885742188

Batch 200: Accu

In [43]:
# Model Testing
def check_accuracy(loader, model):
    model.eval() # Set model into evaluation mode!
    num_samples = 0
    num_correct = 0
    for data, targets in loader:
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        predicted_output = model(data)
        
        _, prediction = predicted_output.max(1)
        num_samples += predicted_output.size(0)
        num_correct += (prediction==targets).sum()

    print(f'Correctly identified samples: {num_correct}')
    print(f'Total samples: {num_samples}')
    print(f'The Validation Accuracy is {num_correct / num_samples * 100.00:.2f}')
    model.train() # Setting model back to training data

In [44]:
print("Checking Accuracy on Training Data...")
check_accuracy(train_loader, model)
print()
print("Checking Accuracy on Testing Data...")
check_accuracy(test_loader, model)

Checking Accuracy on Training Data...
Correctly identified samples: 59603
Total samples: 60000
The Validation Accuracy is 99.34

Checking Accuracy on Testing Data...
Correctly identified samples: 9865
Total samples: 10000
The Validation Accuracy is 98.65
